# Interactive SEIRDS model

This notebook contains code for runnning an SEIRDS model for disease propagation.

We will use the Euler method to numerically sovle coupled first order ODE's for the SEIRDS ODE model.

S = Susceptible, E = Exposed, I = Infected, R = Recovered, D = Dead, N = Total = S+E+I+R+D

$\frac{dS}{dt}=-\frac{q\beta}{N}SI+\tau R$

$\frac{dE}{dt}=\frac{q\beta}{N}SI-\delta E$

$\frac{dI}{dt}=\delta E-(1-\alpha)\gamma I-\alpha\rho I$

$\frac{dR}{dt}=(1-\alpha)\gamma I-\tau R$

$\frac{dD}{dt}=\alpha\rho I$

Here, $\beta$ represents the average number of people that come within infection range of an infected individual per day, which is given by $\frac{1}{\text{number of individuals}}$. $\gamma$ is the remove probability per day, which is given by $\frac{1}{\text{recovery time}}$. $\delta$ is the rate at which exposed individuals become infected (incubation period), which is given by $\frac{1}{\text{incubation period}}$. $\alpha$ is the percentage of infected people who die. $\rho$ is the rate at which people die, given as $\frac{1}{\text{time infected before death}}$. q is the mitigation factor due to quarantining (between 0 and 1), where 0 is perfect quarantine and 1 is no quarantine. $\tau$ is the rate which recovered individuals return to the susceptible population due to loss of immunity.

Toggling the sliders will change the output.

Below we import packages: (MUST RUN)

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
import sys

Next we define a function that has the variables we want to interact with that will output the plot we want to generate. See SEIRDS model notebook for more details on how the function works. Below is the basic SEIRDS model.

In [3]:
@interact(dt=(0.0001,0.1,0.001), tottime=(100,750,5), beta=(0,1,0.01), gamma=(0,1,0.01), rho=(0,1,0.01), delta=(0,1,0.01),
          alpha=(0,1,0.01), tau=(0,1,0.001), q=(0,1,0.01), N=(0,100000,1000), pr=(0,1,0.0001), pe=(0,1,0.0001), pi=(0,1,0.0001))
def SEIRDS_model(dt=0.01, tottime=365, beta=0.25, gamma=0.07, rho=0.05, delta=0.2, alpha=0.05, tau=0.01, q=1, N=50000, pr=0.0, pe=0.0, pi=0.0001):
    #initializing parameters
    #initial percent susceptible
    ps = 1-pr-pe-pi
    
    niter = int(math.ceil(tottime/dt))
    t = np.arange(0, tottime, dt)   
    S = np.zeros(niter)
    E = np.zeros(niter)
    I = np.zeros(niter)
    R = np.zeros(niter)
    D = np.zeros(niter)
    
     
    S[0] = ps*N
    E[0] = pe*N
    I[0] = pi*N
    R[0] = pr*N
    D[0] = 0

    for j in range(niter-1):
        dSdt = -q*beta/N*S[j]*I[j] + tau*R[j]
        dEdt = q*beta/N*S[j]*I[j] - delta*E[j]
        dIdt = delta*E[j] - (1-alpha)*gamma*I[j] - alpha*rho*I[j]
        dRdt = (1-alpha)*gamma*I[j] - tau*R[j]
        dDdt = alpha*rho*I[j]
        S[j+1] = S[j] + dt*dSdt
        E[j+1] = E[j] + dt*dEdt
        I[j+1] = I[j] + dt*dIdt
        R[j+1] = R[j] + dt*dRdt
        D[j+1] = D[j] + dt*dDdt
    N = S + E + I + R + D
    
    plt.plot(t, S)
    plt.plot(t, E)
    plt.plot(t, I)
    plt.plot(t, R)
    plt.plot(t, D)
    plt.plot(t, N)
    plt.legend(['susceptible', 'exposed', 'infected', 'recovered', 'dead', 'total'], loc = 'upper right')
    plt.title('SEIRDS model')
    plt.xlabel('days elapsed since ' + str(pi*100) + ' percent of the population became infected')
    plt.ylabel('population')
    
    print('dt = ' + str(dt), 'tottime = ' + str(tottime), 'beta = ' + str(beta), 'gamma = ' + str(gamma),
      'rho = ' + str(rho), 'delta = ' + str(delta), 'alpha = ' + str(alpha), 'tau = ' + str(tau), 'q = ' + str(q),
          'N[0] = ' + str(N[0]), 'pr = ' +str(pr), 'pe = ' + str(pe), 'pi = ' + str(pi))
   
    return plt.show()

interactive(children=(FloatSlider(value=0.01, description='dt', max=0.1, min=0.0001, step=0.001), IntSlider(va…

# Complete SEIRDS model

We will use the Euler method to numerically sovle coupled first order ODE's for the SIR ODE model.

S = Susceptible, E = Exposed, I = Infected, R = Recovered, D = Dead

S1 = Susceptible Extreme Isolators, S2 = Susceptible Super Spreaders, S3 = Susceptible Average People

E1 = Exposed Extreme Isolators, E2 = Exposed Super Spreaders, E3 = Exposed Average People

I1 = Infected Extreme Isolators, I2 = Infected Super Spreaders, I3 = Infected Average People

R1 = Recovered Extreme Isolators, R2 = Recovered Super Spreaders, R3 = Recovered Average People

D1 = Dead Extreme Isolators, D2 = Dead Super Spreaders, D3 = Dead Average People

$\frac{dS}{dt}=-\frac{q\beta}{N}SI + \tau R$

$\frac{dE}{dt}=\frac{q\beta}{N}SI - \delta E$

$\frac{dI}{dt}=\delta E - (1-\alpha)\gamma I - \alpha\rho I$

$\frac{dR}{dt}=(1-\alpha)\gamma I - \tau R$

$\frac{dD}{dt}=\alpha\rho I$

$\frac{dS_1}{dt}=-\frac{q_1\beta}{N}S_1I + \tau R_1$

$\frac{dS_2}{dt}=-\frac{q_2\beta}{N}S_2I + \tau R_2$

$\frac{dS_3}{dt}=-\frac{q_3\beta}{N}S_3I + \tau R_3$

$\frac{dE_1}{dt}=\frac{q_1\beta}{N}S_1I - \delta E_1$

$\frac{dE_2}{dt}=\frac{q_2\beta}{N}S_2I - \delta E_2$

$\frac{dE_3}{dt}=\frac{q_3\beta}{N}S_3I - \delta E_3$

$\frac{dI_1}{dt}=\delta E_1 - (1-\alpha)\gamma I_1-\alpha\rho I_1$

$\frac{dI_2}{dt}=\delta E_2 - (1-\alpha)\gamma I_2-\alpha\rho I_2$

$\frac{dI_3}{dt}=\delta E_3 - (1-\alpha)\gamma I_3-\alpha\rho I_3$

$\frac{dR_1}{dt}=(1-\alpha)\gamma I_1 - \tau R_1$

$\frac{dR_2}{dt}=(1-\alpha)\gamma I_2 - \tau R_2$

$\frac{dR_3}{dt}=(1-\alpha)\gamma I_3 - \tau R_3$

$\frac{dD_1}{dt}=\alpha\rho I_1$

$\frac{dD_2}{dt}=\alpha\rho I_2$

$\frac{dD_3}{dt}=\alpha\rho I_3$

Here, $\beta$ represents the average number of people that come within infection range of an infected individual per day, which is given by $\frac{1}{\text{number of individuals}}$. $\gamma$ is the remove probability per day, which is given by $\frac{1}{\text{recovery time}}$. $\delta$ is the rate at which exposed individuals become infected (incubation period), which is given by $\frac{1}{\text{incubation period}}$. $\alpha$ is the percentage of infected people who die. $\rho$ is the rate at which people die, given as $\frac{1}{\text{time infected before death}}$. q is the mitigation factor due to quarantining (between 0 and 1), where 0 is perfect quarantine and 1 is no quarantine. $\tau$ is the rate which recovered individuals return to the susceptible population due to loss of immunity.

Toggle the sliders to change the output.

In [7]:
@interact(dt=(0.0001,0.1,0.001), tottime=(100,750,5), beta=(0,1,0.01), gamma=(0,1,0.01), rho=(0,1,0.01), 
          delta=(0,1,0.01), alpha=(0,1,0.01), tau=(0,1,0.0005), N=(0,100000,1000), pr=(0,1,0.0001),
          pe=(0,1,0.001), pi=(0,1,0.001), pss=(0,1,0.01), psd=(0,1,0.1), days_under_first_quarantine=(0,750,5),
         day_second_quarantine_starts=(0,750,5), q11=(0,1,0.05), q12=(0,1,0.05), q13=(0,1,0.05), q21=(0,1,0.05),
          q22=(0,1,0.05), q23=(0,1,0.05), q31=(0,1,0.05), q32=(0,1,0.05), q33=(0,1,0.05), n=(0,8,1))
def SEIRDS_model(dt=0.01, tottime=365, beta=0.25, gamma=0.07, rho=0.05, delta=0.2, alpha=0.05, tau=0.01, N=50000,
              pr=0.0, pe=0.01, pi=0.01, pss=0.2, psd=0.2, days_under_first_quarantine=100,
                day_second_quarantine_starts=200, q11=0, q12=1, q13=0, q21=1, q22=1, q23=1, q31=0.5, q32=1, q33=0.5,
                 n=8):
    #initializing parameters
    #initial percent susceptible
    ps = 1-pr-pe-pi
    
    
    
    
    niter = int(math.ceil(tottime/dt))
    t = np.arange(0, tottime, dt)  
    
    S = np.zeros(niter) # total susceptible population
    S1 = np.zeros(niter) # susceptible extreme isolators
    S2 = np.zeros(niter) # susceptible super spreaders
    S3 = np.zeros(niter) # susceptible average people
    E = np.zeros(niter) # total exposed population
    E1 = np.zeros(niter) # exposed extreme isolators
    E2 = np.zeros(niter) # exposed super spreaders
    E3 = np.zeros(niter) # exposed average people
    I = np.zeros(niter) # total infected population
    I1 = np.zeros(niter) # infected extreme isolators
    I2 = np.zeros(niter) # infected super spreaders
    I3 = np.zeros(niter) # infected average people
    R = np.zeros(niter) # total recovered population
    R1 = np.zeros(niter) # recovered extreme isolators
    R2 = np.zeros(niter) # recovered super spreaders
    R3 = np.zeros(niter) # recovered average people
    D = np.zeros(niter) # total dead population
    D1 = np.zeros(niter) # dead extreme isolators
    D2 = np.zeros(niter) # dead super spreaders
    D3 = np.zeros(niter) # dead average people
     
    S[0] = ps*N
    S1[0] = psd*S[0]
    S2[0] = pss*S[0]
    S3[0] = S[0]-S1[0]-S2[0]
    E[0] = pe*N
    E1[0] = psd*E[0]
    E2[0] = pss*E[0]
    E3[0] = E[0]-E1[0]-E2[0]
    I[0] = pi*N
    I1[0] = psd*I[0]
    I2[0] = pss*I[0]
    I3[0] = I[0]-I1[0]-I2[0]
    R[0] = pr*N
    R1[0] = psd*R[0]
    R2[0] = pss*R[0]
    R3[0] = R[0]-R1[0]-R2[0]
    D[0] = 0
    D1[0] = 0
    D2[0] = 0
    D3[0] = 0
    
    for j in range(niter-1):
        if j <= days_under_first_quarantine / dt:
            q1 = q11 # initial mitigation factor for extreme isolators
            q2 = q21 # initial mitigation factor for super spreaders
            q3 = q31 # initial mitigation factor for average people
        elif j <= day_second_quarantine_starts / dt and j > days_under_first_quarantine / dt:
            q1 = q12 # second mitigation factor for extreme isolators
            q2 = q22 # second mitigation factor for super spreaders
            q3 = q32 # second mitigation factor for average people
        else:
            q1 = q13 # final mitigation factor for extreme isolators
            q2 = q23 # final mitigation factor for super spreaders
            q3 = q33 # final mitigation factor for average people
        dS1dt = -q1*beta/N*S1[j]*I[j] + tau*R1[j]
        dS2dt = -q2*beta/N*S2[j]*I[j] + tau*R2[j]
        dS3dt = -q3*beta/N*S3[j]*I[j] + tau*R3[j]
        dEdt = q1*beta/N*S1[j]*I[j] + q2*beta/N*S2[j]*I[j] + q3*beta/N*S3[j]*I[j] - delta*E[j]
        dE1dt = q1*beta/N*S1[j]*I[j] - delta*E1[j]
        dE2dt = q2*beta/N*S2[j]*I[j] - delta*E2[j]
        dE3dt = q3*beta/N*S3[j]*I[j] - delta*E3[j]
        dIdt = delta*E[j] - (1-alpha)*gamma*I[j] - alpha*rho*I[j]
        dI1dt = delta*E1[j] - (1-alpha)*gamma*I1[j] - alpha*rho*I1[j]
        dI2dt = delta*E2[j] - (1-alpha)*gamma*I2[j] - alpha*rho*I2[j]
        dI3dt = delta*E3[j] - (1-alpha)*gamma*I3[j] - alpha*rho*I3[j]
        dRdt = (1-alpha)*gamma*I[j] - tau*R[j]
        dR1dt = (1-alpha)*gamma*I1[j] - tau*R1[j]
        dR2dt = (1-alpha)*gamma*I2[j] - tau*R2[j]
        dR3dt = (1-alpha)*gamma*I3[j] - tau*R3[j]
        dDdt = alpha*rho*I[j]
        dD1dt = alpha*rho*I1[j]
        dD2dt = alpha*rho*I2[j]
        dD3dt = alpha*rho*I3[j]
        S1[j+1] = S1[j] + dt*dS1dt
        S2[j+1] = S2[j] + dt*dS2dt
        S3[j+1] = S3[j] + dt*dS3dt
        E[j+1] = E[j] + dt*dEdt
        E1[j+1] = E1[j] + dt*dE1dt
        E2[j+1] = E2[j] + dt*dE2dt
        E3[j+1] = E3[j] + dt*dE3dt
        I[j+1] = I[j] + dt*dIdt
        I1[j+1] = I1[j] +dt*dI1dt
        I2[j+1] = I2[j] +dt*dI2dt
        I3[j+1] = I3[j] +dt*dI3dt
        R[j+1] = R[j] + dt*dRdt
        R1[j+1] = R1[j] + dt*dR1dt
        R2[j+1] = R2[j] + dt*dR2dt
        R3[j+1] = R3[j] + dt*dR3dt
        D[j+1] = D[j] + dt*dDdt
        D1[j+1] = D1[j] + dt*dD1dt
        D2[j+1] = D2[j] + dt*dD2dt
        D3[j+1] = D3[j] + dt*dD3dt
    S = S1 + S2 + S3
    N = S + E + I + R + D
    T1 = S1 + E1 + I1 + R1 + D1
    T2 = S2 + E2 + I2 + R2 + D2
    T3 = S3 + E3 + I3 + R3 + D3

    if n == 0:
        plt.plot(t, S)   
        plt.plot(t, S3)
        plt.plot(t, S1)
        plt.plot(t, S2)
        plt.legend(['Total susceptible', str(round(((1-pss-psd)*100),3)) + '% "average" people',
                    str(round((psd*100),3)) + '% extreme isolators',
                    str(round((pss*100),3)) + '% super spreaders'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Susceptible population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    elif n == 1:
        plt.plot(t, E)
        plt.plot(t, E3)
        plt.plot(t, E1)
        plt.plot(t, E2)
        plt.legend(['Total exposed', str(round(((1-pss-psd)*100),3)) + '% "average" people',
                    str(round((psd*100),3)) + '% extreme isolators',
                    str(round((pss*100),3)) + '% super spreaders'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Exposed population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    elif n == 2:
        plt.plot(t, I)
        plt.plot(t, I3)
        plt.plot(t, I1)
        plt.plot(t, I2)
        plt.legend(['Total infected', str(round(((1-pss-psd)*100),3)) + '% "average" people',
                    str(round((psd*100),3)) + '% extreme isolators',
                    str(round((pss*100),3)) + '% super spreaders'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Infected population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()    
    elif n == 3:
        plt.plot(t, R)
        plt.plot(t, R3)
        plt.plot(t, R1)
        plt.plot(t, R2)
        plt.legend(['Total recovered', str(round(((1-pss-psd)*100),3)) + '% "average" people',
                    str(round((psd*100),3)) + '% extreme isolators',
                    str(round((pss*100),3)) + '% super spreaders'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Recovered population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    elif n == 4:
        plt.plot(t, D)
        plt.plot(t, D3)
        plt.plot(t, D1)
        plt.plot(t, D2)
        plt.legend(['Total dead', str(round(((1-pss-psd)*100),3)) + '% "average" people',
                    str(round((psd*100),3)) + '% extreme isolators',
                    str(round((pss*100),3)) + '% super spreaders'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Dead population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    elif n == 5:
        plt.plot(t, S1)
        plt.plot(t, E1)
        plt.plot(t, I1)
        plt.plot(t, R1)
        plt.plot(t, D1)
        plt.plot(t, T1)
        plt.legend(['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Dead', 'Total'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Extreme isolators)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()    
    elif n == 6:
        plt.plot(t, S2)
        plt.plot(t, E2)
        plt.plot(t, I2)
        plt.plot(t, R2)
        plt.plot(t, D2)
        plt.plot(t, T2)
        plt.legend(['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Dead', 'Total'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Super spreaders)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    elif n == 7:
        plt.plot(t, S3)
        plt.plot(t, E3)
        plt.plot(t, I3)
        plt.plot(t, R3)
        plt.plot(t, D3)
        plt.plot(t, T3)
        plt.legend(['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Dead', 'Total'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days ("Average" people)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()    
    else:
        plt.plot(t, S)
        plt.plot(t, E)
        plt.plot(t, I)
        plt.plot(t, R)
        plt.plot(t, D)
        plt.plot(t, N)
        plt.legend(['Susceptible', 'Exposed', 'Infected', 'Recovered', 'Dead', 'Total'], loc='upper right')
        plt.title('Mixed SEIRDS model for lifting restrictions after ' + str(int(days_under_first_quarantine))
                + ' days and  \n reinstating restrictions after '
                + str(int(day_second_quarantine_starts-days_under_first_quarantine))
                + ' days (Total population)')
        plt.xlabel('days elapsed since 1 percent of the population became infected')
        plt.ylabel('population')
        plt.show()
    
    print('dt = ' + str(dt) + ',', 'tottime = ' + str(tottime) + ',', 'beta = ' + str(beta) + ',',
          'gamma = ' + str(gamma) + ',','rho = ' + str(rho) + ',', 'delta = ' + str(delta) + ',',
          'alpha = ' + str(alpha) + ',', 'tau = ' + str(tau) +',', 'N[0] = ' + str(N[0]) + ',',
          'pr = ' +str(pr) + ',', 'pe = ' + str(pe) + ',', 'pi = ' + str(pi) + ',',
         'days quarantined 1 = ' + str(days_under_first_quarantine) + ',', 'day resume quarantine = ' 
         + str(day_second_quarantine_starts) + ',', 'q11 = ' + str(q11) + ',', 'q12 = ' + str(q12) + ',',
         'q13 = ' + str(q13) + ',', 'q21 = ' + str(q21) + ',', 'q22 = ' + str(q22) + ',', 'q23 = ' + str(q23) + ',',
         'q31 = ' + str(q31) + ',', 'q32 = ' + str(q32) + ',', 'q33 = ' + str(q33) + '.')
   
    return plt.show()

interactive(children=(FloatSlider(value=0.01, description='dt', max=0.1, min=0.0001, step=0.001), IntSlider(va…

$n=0$ $\rightarrow$ susceptible population, $n=1$ $\rightarrow$ exposed population, $n=2$ $\rightarrow$ infected population,
$n=3$ $\rightarrow$ recovered population, $n=4$ $\rightarrow$ dead population, 

$n=5$ $\rightarrow$ extreme isolators, $n=6$ $\rightarrow$ super spreaders, $n=7$ $\rightarrow$ average people, $n=8$ $\rightarrow$ total population